In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",  one_hot=True)
global_step = tf.Variable(0, trainable=False, name='global_step')

#변수들
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
W1 = tf.Variable(tf.random_normal([784, 100]))
W2 = tf.Variable(tf.random_normal([100, 10]))
b1 = tf.Variable(tf.random_normal([100]))
b2 = tf.Variable(tf.random_normal([10]))

#모델
L1 = tf.matmul(X, W1) + b1
S1 = tf.nn.relu(L1)
model =  tf.matmul(L1, W2) + b2
 
cost =   tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))                        
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost, global_step=global_step)      #

#검증을 위한 코드/모니터링
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)     
tf.summary.histogram("W1", W1)     
tf.summary.histogram("b1", b1)

#가중치 저장
saver = tf.train.Saver(tf.global_variables())               #
ckpt = tf.train.get_checkpoint_state('./model')       #   

#파라미터
training_epoch = 5
batch_size = 100

#세션런
with tf.Session() as sess:
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):    #
        saver.restore(sess, ckpt.model_checkpoint_path)                                   #
    else:                                                                                                                        #
        sess.run(tf.global_variables_initializer())                                                   #                          
        
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/ckpt', sess.graph)    
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch =  int(mnist.train.num_examples / batch_size)   
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
            avg_cost += c / total_batch
            
            summary = sess.run(merged, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 1})
            writer.add_summary(summary, global_step=sess.run(global_step))
            
        print('Step:', '%03d '%sess.run(global_step),' Cost:', '%.4f'%(avg_cost), ' Acc:', 
              sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))
    
    saver.save(sess, './model/dnn.ckpt', global_step=global_step)           #

Extracting ./mnist/data/train-images-idx3-ubyte.gz


Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f836c0fbfd0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f836c0f9a20>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f836c0f98d0>)


INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-11000


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Variable_4 not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-2-3d2e1d741d0a>:34)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr